In [2]:
from gensim.models import KeyedVectors
# 学習データの読み込み
recipe_wv = KeyedVectors.load_word2vec_format('../data/trained_data/mix_1_9_600_5.vec.pt', binary=True)
recipe2_wv = KeyedVectors.load_word2vec_format('../data//trained_data/GloVe/input_glove_vector_5_600.vec.pt', binary=False)

# 類推するワード
search_word ="熱湯"

# 学習した単語に近しい単語の意味を類推
try:
  recipe_results = recipe_wv.most_similar(positive=[search_word])
except KeyError as e:
  print(e)
  recipe_results = ["単語が存在しない"]

try:
  recipe2_results = recipe2_wv.most_similar(positive=[search_word])
except KeyError as e:
  print(e)
  recipe2_results = ["単語が存在しない"]

# 結果の表示
print("\nWord2vecの学習データ\n")
for result in recipe_results:
  print(result)

print("\nGloVeの学習データ\n")
for result in recipe2_results:
  print(result)


Word2vecの学習データ

('お湯', 0.6482459902763367)
('湯', 0.47677311301231384)
('熱闘', 0.4546242356300354)
('湯通し', 0.42026081681251526)
('湯に', 0.4171830713748932)
('沸かし', 0.3864058256149292)
('ぬるま湯', 0.36578798294067383)
('ねっとう', 0.36243024468421936)
('冷水', 0.34496891498565674)
('温水', 0.33779671788215637)

GloVeの学習データ

('お湯', 0.6346032619476318)
('湯', 0.6016823053359985)
('湯通し', 0.5449549555778503)
('沸かし', 0.5248202681541443)
('さっと', 0.5136610865592957)
('ダイエットサポートティーバッグ', 0.5031768679618835)
('茹でる', 0.4856308698654175)
('ゆがい', 0.4739053249359131)
('サッと', 0.4663752317428589)
('茹で', 0.4507855176925659)


In [8]:
f = open("../data/rakuten_url.txt",'r')
data = f.read()
f.close()

data = data.split(',')

reg = int(len(data)/10)
print(len(data))

sep_data = [ data[i:i+reg] for i in range(0,len(data),reg)]

sep_num = 1
b_name = "../data/sep_scry_data/rakuten-url"
for sep_urls in sep_data:
    file_name = b_name + str(sep_num) + ".txt"
    print(file_name,":",len(sep_urls))
    sep_f = open(file_name,"w",encoding='utf-8')
    sep_f.write(",".join(sep_urls))
    sep_f.close()
    sep_num += 1


1381089
../data/sep_scry_data/rakuten-url1.txt : 138108
../data/sep_scry_data/rakuten-url2.txt : 138108
../data/sep_scry_data/rakuten-url3.txt : 138108
../data/sep_scry_data/rakuten-url4.txt : 138108
../data/sep_scry_data/rakuten-url5.txt : 138108
../data/sep_scry_data/rakuten-url6.txt : 138108
../data/sep_scry_data/rakuten-url7.txt : 138108
../data/sep_scry_data/rakuten-url8.txt : 138108
../data/sep_scry_data/rakuten-url9.txt : 138108
../data/sep_scry_data/rakuten-url10.txt : 138108
../data/sep_scry_data/rakuten-url11.txt : 9
